# TO DO: put this in run script...

In [1]:
import numpy as np
import torch
from mcspace.model import MCSPACE
from mcspace.trainer import train_model
from mcspace.data_utils import get_data, get_mouse_diet_perturbations_dataset
from mcspace.utils import get_device, pickle_load, pickle_save,\
    estimate_process_variance, MODEL_FILE, DATA_FILE, \
    get_mcspace_cooccur_prob, get_sig_perturbation_diff_subset, \
    get_posterior_summary_data, get_pert_otu_sub, get_min_loss_path, apply_taxonomy_threshold, \
    get_summary_results
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import mcspace.visualization as vis
from mcspace.dataset import DataSet
import pandas as pd

In [2]:
pwd

'C:\\Users\\Gary\\Partners HealthCare Dropbox\\Gurdip Uppal\\research_bwh\\MCSPACE_FINAL\\MCSPACE\\mcspace\\paper\\human_runs'

# Paths

In [3]:
rootpath = Path("../../")
basepath = Path("./")

In [4]:
runpath = rootpath / "paper" / "human_runs" / "runs"

In [5]:
treepath = rootpath / "data" / "human_experiments" / "human_otus_tree" / "output" # *** rename this
treefile = "newick_tree_query_reads.nhx"

In [6]:
outpath = basepath / "results"
outpath.mkdir(exist_ok=True, parents=True)

In [8]:
# ls

# Load best model

In [9]:
respath = get_min_loss_path(runpath, np.arange(10))

c:\users\gary\partners healthcare dropbox\gurdip uppal\research_bwh\mcspace_final\mcspace\mcspace\utils.py:415: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.l

0
1
2
3
4
5
6
7
8
9
3


In [10]:
model = torch.load(respath / MODEL_FILE)
data = pickle_load(respath / DATA_FILE)

taxonomy_temp = pickle_load(respath / "taxonomy.pkl")

times = list(data['count_data'].keys())
subjects = list(data['count_data'][1].keys())
num_otus = data['count_data'][times[0]][subjects[0]].shape[1]
num_times = len(times)
num_subjects = len(subjects)

C:\Users\Gary\AppData\Local\Temp\ipykernel_122388\2930395400.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(respath / MODEL_FILE)


### load final taxonomy

In [11]:
taxfile =  rootpath / "data" / "human_experiments" / "gappa_taxonomy" / "human_taxonomy.csv"
finaltax = pd.read_csv(taxfile, index_col=0)

In [12]:
taxonomy_temp.head()

,Kingdom,Phylum,Class,Order,Family,Genus
Otu,,,,,,
Otu121,Bacteria,Firmicutes,Bacilli,Erysipelotrichales,Erysipelatoclostridiaceae,na
Otu14,Bacteria,Firmicutes,Clostridia,Lachnospirales,Lachnospiraceae,na
Otu150,Bacteria,Firmicutes,Clostridia,Oscillospirales,Oscillospiraceae,uncultured
Otu37,Bacteria,Bacteroidota,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides
Otu6,Bacteria,Actinobacteriota,Actinobacteria,Bifidobacteriales,Bifidobacteriaceae,Bifidobacterium


In [13]:
taxlist = list(taxonomy_temp.index)

In [14]:
taxonomy = finaltax.loc[taxlist,:]

## Get summary results

In [15]:
thetadf, betadf, pertsdf = get_posterior_summary_data(model, data, taxonomy, times, subjects)

### get relative abundance data

In [16]:
def get_bulk_relative_abundances(reads, times, subjects, taxonomy):
    num_otus = taxonomy.shape[0]
    multiind = pd.MultiIndex.from_frame(taxonomy)
    ntime = len(times)
    
    radfs = {}
    for s in subjects:
        ra = np.zeros((ntime, num_otus))
        for i,t in enumerate(times):
            counts = reads[t][s].cpu().detach().clone().numpy()
            rabun = counts/counts.sum(axis=1, keepdims=True)
            bulk_rabun = np.mean(rabun, axis=0)
            ra[i,:] = bulk_rabun
        df = pd.DataFrame(data=ra.T, index=multiind, columns=times)
        radfs[s] = df
    return radfs

In [17]:
reads = data['count_data'] #.cpu().detach().clone().numpy()
bulktemp = get_bulk_relative_abundances(reads, times, subjects, taxonomy.reset_index())

In [18]:
bulk = bulktemp['H11'].reset_index()[['Otu'] + times].set_index('Otu')

In [19]:
bulk.head()

,1,2,3,4,5
Otu,,,,,
Otu121,0.006974,0.007406,0.000576,0.002787,0.001610
Otu14,0.020579,0.078228,0.025125,0.103553,0.044628
Otu150,0.000119,0.028550,0.000008,0.000006,0.000003
Otu37,0.001440,0.003773,0.002748,0.006807,0.004576
Otu6,0.019864,0.006888,0.062092,0.010495,0.004887


# Output results

In [23]:
# pertsdf

In [24]:
thetadf.to_csv(outpath / "assemblages.csv")
betadf.to_csv(outpath / "assemblage_proportions.csv")

In [25]:
bulk.to_csv(outpath / "relative_abundances.csv")

In [27]:
taxonomy.to_csv(outpath / "taxonomy.csv")